# 0. Préparation de l'environnement de travail

Pour le bon déroulé du projet, on installe les packages suivants : 
- requests : pour la récupération des données par API
- pandas : pour la création et la manipulation de tableaux de données
- zipfile : pour ouvrir, lire ou extraire des données compressées
- os : pour interagir avec le système de fichiers

In [ ]:
import requests
import pandas as pd
import zipfile
import os

# 1. Importation des données

Des API étaient disponibles pour l'ensemble de nos jeux de données, mais nous avons préféré les télécharger en brut et les mettre en cache local sur S3 pour travailler à partir de là, afin de favoriser la reproductibilité du projet. 

## a. Elections présidentielles

Afin d'étudier le vote RN dans les communes françaises, nous avons choisi de nous concentrer sur les résultats à la présidentielle 2022 du premier tour, le 10 avril 2022. Nous avions initialement pensé à étudier les résultats au premier tour des législatives 2024, plus récents, mais avons réalisé que bon nombre de circonscriptions n'avaient pas de candidats RN, ce qui aurait pû créer un biais. Avec les présidentielles, nous avions l'assurance d'obtenir des résultats pour chaque commune. 

Entre le premier et le second tour de la présidentielle, nous avons naturellement porté notre attention sur les résultats du premier tour, puisqu'il y avait plus de candidats. 

> Remarque : peut-être détailler un peu plus

Les résultats des élections en France sont disponibles sur **[data.gouv.fr](https://www.data.gouv.fr/datasets/election-presidentielle-des-10-et-24-avril-2022-resultats-definitifs-du-1er-tour/)**. 
Malheureusement, le niveau communal n'est pas proposé, donc nous avons choisi de télécharger le niveau **sous-communal** dans l'optique d'agréger les résultats par la suite. 

In [ ]:
# DONNÉES INFRASTRUCTURES (S3 SSPCloud)

# Chemin S3 du fichier
path_infra_s3 = "s3://hantier/DS_BPE_2024_data.csv"

# Lecture directe depuis S3
df_infrastruct = pd.read_csv("s3://hantier/DS_BPE_2024_data.csv", sep=';')  # le séparateur utilisé est un ;
print("Le dataframe df_infrastruct peut être traité comme un data frame classique avec pandas")

## b. Infrastructures sportives et culturelles


Afin que cette étape se déroule bien, il faut récupérer la base DS_BPE_2024_data.csv préalablement importée sur S3.

In [ ]:
url_elect = "https://www.data.gouv.fr/api/1/datasets/r/6d9b33e5-667d-4c3e-9a0b-5fdf5baac708"
path_elect = "/home/onyxia/work/Projet-Python-S3/code/results_elect.xlsx"

# Téléchargement du fichier Excel
response = requests.get(url_elect)
response.raise_for_status()

with open(path_elect, "wb") as f:
    f.write(response.content)

# Lecture du fichier Excel
df_extrait = pd.read_excel(path_elect, sheet_name=0)

print("Le dataframe df_extrait peut être traité comme un data frame classique avec pandas")

print("Informations sur le dataframe df_extrait qui est notre df initial")
print(df_extrait.columns)
print(len(df_extrait))


# la colonne unamed 53 correspond au nombre de voix de JLF, la colonne unamed 67 correspond à celui de marine le pen
liste_var = ['Code du département', 
    'Libellé du département', 
    'Code de la commune', 
    'Libellé de la commune',
    'Inscrits', 
    'Abstentions',
    '% Abs/Ins',
    'Unnamed: 53', 
    'Unnamed: 67']
df_extrait2 = df_extrait[liste_var]
df_extrait3 = df_extrait2.rename(columns={'Unnamed: 53': 'score_rn'})
df_elect = df_extrait3.rename(columns={'Unnamed: 67': 'score_lfi'})
print("Informations sur le dataframe df_elect")
print(df_elect.columns)
print(len(df_elect))
print(df_elect.head(4))

2) Préparation des données infrastructures
Dans un second temps, nous devons préparer le dataframe des infrastructures. Nous allons le passer d'un format long à un format large, autrement dit, au lieu d'avoir une ligne par infrastructure, nous aurons une ligne par ville et une colonne par infrastructure.

In [ ]:
print("Informations sur le dataframe df_infrastruct")
print(df_infrastruct.columns)
print(df_infrastruct.size)
print(df_infrastruct.head())

In [ ]:
df_infra_large = (
    df_infrastruct
    .query("TIME_PERIOD == 2024")
    .groupby(["GEO", "FACILITY_TYPE"], as_index=False)["OBS_VALUE"]
    .sum()
    .pivot(index="GEO", columns="FACILITY_TYPE", values="OBS_VALUE")
    .fillna(0)
    .reset_index()
)


In [ ]:
print("Vérifications sur le dataframe df_infra_large")
print(df_infra_large.columns)
print(df_infra_large.size)
print(df_infra_large.head())

# 2. Statistiques descriptives 

Cette partie a pour but de vérifier que les bases sont propres et utilisables correctement. Les statistiques descriptives les statistiques descriptives permettront d'être sûres qu'il n'y a pas de données en double, de données manquantes ou de données aberrantes. 

In [ ]:
import geopandas as gpd
import matplotlib.pyplot as plt

# a. Elections présidentielles

Nous allons représenter les résultats aux élections présidentielles des deux partis choisis. Pour ce faire, dans un premier temps, il faut calculer 

In [ ]:
print("Le score moyen par commune du RN était")
print(df_elect["score_rn"].mean())
print(df_elect["score_rn"].median())
print("Le score moyen par commune de LFI était")
print(df_elect["score_lfi"].mean())
print(df_elect["score_lfi"].median())

Si on compare aux scores nationaux, on obtient des chiffres assez proches (Mme Le Pen ayant obtenu 23,15 % des voix au premier tour). La différence entre les deux chiffres peut être due au fait que nous avons fait une moyenne par commune sans pondérer par le nombre d'habitants, ce qui fait que les petites communes, plutôt en campagne et qui votent plus largement pour le RN sur sur-représentées. 

In [ ]:
# Préparation du code département : il faut le formater pour qu'il corresponde aux données des fonds de carte

df_elect['Code du département'] = (
    df_elect['Code du département']
    .astype(str)
    .str.zfill(2)
)

# Agrégation par département

# Moyenne des scores communaux

df_dep = (
    df_elect
    .groupby(['Code du département', 'Libellé du département'], as_index=False)
    .agg({
        'score_rn': 'mean',
        'score_lfi': 'mean'
    })
)


# Chargement du fond de carte départements

gdf_dep = gpd.read_file(
    "https://france-geojson.gregoiredavid.fr/repo/departements.geojson"
)

gdf_dep['code'] = gdf_dep['code'].astype(str).str.zfill(2)

# Fusion de la carte avec les données

carte_dep = gdf_dep.merge(
    df_dep,
    left_on='code',
    right_on='Code du département',
    how='left'
)

# Carte RN par département

fig, ax = plt.subplots(figsize=(10, 10))

carte_dep.plot(
    column='score_rn',
    cmap='Blues',
    legend=True,
    edgecolor='black',
    ax=ax
)

ax.set_title("Score Mme Le Pen par département (1er tour 2022)")
ax.axis('off')
plt.show()

# Carte LFI par département

fig, ax = plt.subplots(figsize=(10, 10))

carte_dep.plot(
    column='score_lfi',
    cmap='Reds',
    legend=True,
    edgecolor='black',
    ax=ax
)

ax.set_title("Score M. Mélenchon par département (1er tour 2022)")
ax.axis('off')
plt.show()


Les cartes suivantes sont cohérentes avec les statistiques publiées à propos des élections. Cependant, nous rappelons qu'ici, notre variable d'intérêt est la commune, c'est pour cela que nous n'avons pas pondéré par le nombre d'habitants donc les résultats publiés en ligne. Toutefois, nous ne voyons pas d'incohérence majeure.